In [1]:
import pandas as pd
import os

In [2]:
# Combine separate airline sheets into one table. All sheets have identical columns.
files = ['airline_destination_data/'+x for x in os.listdir('airline_destination_data/') if ".csv" in x]
airline_table = pd.DataFrame(columns=['Airline', 'Notes', 'icao'])
for file in files:
    airline_table = airline_table.append(pd.read_csv(file))

In [3]:
# ICAO column is named 'ident' in the geocoded table.
airline_table = airline_table.rename(columns={'icao':'ident'})

# Join airports and airlines on ICAO code. This will generate duplicate rows that we will use to fill the Y/N table.
airports_with_airlines = pd.merge(pd.read_csv('airport_geodata/airports.csv'), airline_table, on=['ident'])

In [4]:
# Create a new column for each airline, with N value by default.
for airline in airports_with_airlines.Airline.unique():
    airports_with_airlines[airline] = "N"

In [5]:
# Mark Y in each airline's column if they reach each airport.
for airport in airports_with_airlines.name.unique():
    airport_frame = airports_with_airlines[airports_with_airlines.name == airport]
    for airline in airport_frame.Airline.unique():
        for idx in airport_frame.index:
            airports_with_airlines[airline][idx] = "Y"

# Now that there is a column for each airline, we can drop duplicates by airport.
airports_with_airlines = airports_with_airlines.drop_duplicates(subset=['name']).reset_index()

C:\Users\jgmm2\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
# Count the number of airlines that reach each airport.
airports_with_airlines['airline_count'] = 0
for row in airports_with_airlines.index:
    airports_with_airlines.loc[row, 'airline_count'] = \
    airports_with_airlines.iloc[row].tolist().count("Y")

In [7]:
# Reorganize columns for readability
airports_with_airlines = airports_with_airlines[['id', 'ident',
       'type', 'name', 'latitude_deg', 'longitude_deg', 'elevation_ft',
       'continent', 'iso_country', 'iso_region', 'municipality','Alaska Airlines', 'American Airlines',
       'Allegiant Air', 'Frontier Airlines', 'JetBlue Airways',
       'Delta Air Lines', 'United Airlines', 'SkyWest Airlines',
       'Southwest Airlines', 'Hawaiian Airlines', 'airline_count']]
airports_with_airlines

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,...,American Airlines,Allegiant Air,Frontier Airlines,JetBlue Airways,Delta Air Lines,United Airlines,SkyWest Airlines,Southwest Airlines,Hawaiian Airlines,airline_count
0,123,BIKF,large_airport,Keflavik International Airport,63.985001,-22.605600,171.0,EU,IS,IS-2,...,N,N,N,N,Y,Y,N,N,N,2
1,1321,CNM2,small_airport,Melbourne Airport,42.824200,-81.548599,735.0,NaN,CA,CA-ON,...,N,N,N,N,N,Y,N,N,N,1
2,1717,CYEG,large_airport,Edmonton International Airport,53.309700,-113.580002,2373.0,NaN,CA,CA-AB,...,N,N,N,N,Y,Y,Y,N,N,3
3,1840,CYOW,large_airport,Ottawa Macdonald-Cartier International Airport,45.322498,-75.669197,374.0,NaN,CA,CA-ON,...,N,N,N,N,N,N,Y,N,N,1
4,1928,CYUL,large_airport,Montreal / Pierre Elliott Trudeau Internationa...,45.470600,-73.740799,118.0,NaN,CA,CA-QC,...,N,N,N,N,Y,N,Y,N,N,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,27145,YSSY,large_airport,Sydney Kingsford Smith International Airport,-33.946098,151.177002,21.0,OC,AU,AU-NSW,...,N,N,N,N,Y,Y,N,N,Y,3
468,27188,ZBAA,large_airport,Beijing Capital International Airport,40.080101,116.584999,116.0,AS,CN,CN-11,...,N,N,N,N,N,Y,N,N,N,1
469,330820,ZBAD,large_airport,Beijing Daxing International Airport,39.509945,116.410920,98.0,AS,CN,CN-13,...,N,N,N,N,Y,N,N,N,N,1
470,27223,ZSPD,large_airport,Shanghai Pudong International Airport,31.143400,121.805000,13.0,AS,CN,CN-31,...,N,N,N,N,Y,Y,N,N,N,2


In [8]:
airports_with_airlines.to_csv('airport_geodata/airline_master_yntable.csv', index=False)